### 通过张正友标定法获取相机参数
#### 步骤
```
1.准备标定板：使用一个已知尺寸的标定板（通常是黑白棋盘格），并在标定板上标记出若干个特征点。
2.拍摄图像：将标定板放置在不同的位置姿态下拍摄多张图像，并记录每张图像对应的相机姿态（即相机位姿）。
3.提取角点：对每张图像进行角点提取，即找到标定板上的特征点在图像中的对应位置。
4.计算相机姿态：通过已知的标定板尺寸和提取的角点，计算每张图像对应的相机位姿。
5.标定相机：使用相机标定公式，计算出相机的内参矩阵和畸变系数。
6.评估标定结果：使用标定结果评估指标（如重投影误差）来评估标定结果的准确性。
需要注意的是，该方法需要至少使用10张以上的图像才能稳健地标定相机。此外，标定板的布局和大小也会对标定结果产生影响。
```
#### 注意
```
1.标定板的质量：标定板需要具有较高的精度和稳定性，以保证提取的角点准确无误。同时，在摄影过程中需要避免标定板发生变形或损坏。
2.角点提取算法：角点提取算法需要能够精确地检测到标定板上的特征点，并且对噪声和光照变化具有一定的鲁棒性。目前常用的角点提取算法包括Harris角点算法、Shi-Tomasi算法等。
3.图像采集：在采集图像时，应尽可能保持相机姿态的多样性和分布均匀性，以覆盖尽可能广泛的空间。
4.相机位姿的估计：相机位姿的估计需要具有一定的精度和可靠性。通常可以使用三维-二维点对求解相机姿态，也可以使用直接法等方法进行位姿估计。
5.重投影误差的评估：重投影误差是评估标定结果好坏的一个重要指标。在评估时，需要注意将误差控制在一定范围内，同时了解误差来源，以便对标定结果进行进一步优化。
6.畸变校正的效果：畸变是相机成像过程中不可避免的问题，需要进行畸变校正。在进行畸变校正时，需要注意调整参数以达到最佳校正效果，并对校正结果进行评估和验证。
```


In [123]:
# 导包
import os
import cv2
import numpy as np

In [124]:
# 定义棋盘格模板规格:只算内角点个数,不算最外面的一圈点
pattern_size = (11, 8)
# 定义每个棋盘格的物理尺寸（单位：毫米）
square_size = 40.0
# 图像所在文件夹的位置
original_images = './CalibrationPlate/original/'   # 原始图片保存位置
resize_images = './CalibrationPlate/resize/'   # 调整尺寸后的图像保存位置
corner_images = './CalibrationPlate/corner/'   # 显示角点的图像保存位置

In [125]:
"""_调整图片大小,防止图片过大引起崩溃_
Args:
    input (_string_): _输入图片的文件夹路径_
    output (_string_): _输出图片的文件夹路径_
    width (_int_): _输出图片宽度_
    height (_int_): _输出图片高度_
"""
def ResizeImage(input, output, width, height):
    Images = os.listdir(input)
    for fname in Images:
        image = cv2.imread(input + fname)
        out = cv2.resize(image, (width, height))
        cv2.imwrite(output + fname, out)

In [127]:
# 调整图片尺寸
new_width = 800
new_height = 600
ResizeImage(original_images, resize_images, new_width, new_height)

In [ ]:
# 世界坐标系中的棋盘格点,例如(0,0,0),(2,0,0)...(8,5,0)，去掉Z坐标，记为二维矩阵
world_point = np.zeros((pattern_size[0] * pattern_size[1], 3), np.float32)
# 将世界坐标系建在标定板上，所有点的Z坐标全部为0，所以只需要赋值x和y
world_point[:, :2] = np.mgrid[0:pattern_size[0]*square_size:square_size,
                              0:pattern_size[1]*square_size:square_size].T.reshape(-1, 2)
# 储存棋盘格角点的世界坐标和图像坐标对
world_points = []  # 世界坐标系中的三维点
image_points = []  # 图像平面的二维点

In [ ]:
"""
角点精准化迭代过程的终止条件:
第一项:表示迭代次数达到最大次数时停止迭代;
第二项:表示角点位置变化的最小值已经达到最小时停止迭代;
第三项和第四项：表示设置寻找亚像素角点的参数--采用的停止准则是最大循环次数30和最大误差容限0.001
"""
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER,
            30, 0.001)

In [ ]:
images = os.listdir(resize_images)   # 读入图像序列
index = 0
for fname in images:
    image = cv2.imread(resize_images + '/' + fname)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)   # RGB转灰度
    # 寻找棋盘格角点,存放角点于corners中
    # 如果找到足够点对,将其存储起来,ret为非零值
    ret, corners = cv2.findChessboardCorners(gray, pattern_size, None)
    # 检测到角点后,进行亚像素级别角点检测,更新角点

    if ret == True:
        index += 1
        # 输入图像gray;角点初始坐标corners;搜索窗口为2*winsize+1;表示窗口的最小(-1.-1)表示忽略;求角点的迭代终止条件
        cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        world_points.append(world_point)   # 世界坐标
        image_points.append(corners)  # 图像坐标
        cv2.drawChessboardCorners(image, pattern_size, corners, ret)
        cv2.imwrite(corner_images + '/corners_' + str(index) + '.jpg', image)
        cv2.waitKey(10)
    cv2.destroyAllWindows()

In [ ]:
"""
输入:世界坐标系里的位置;像素坐标;图像的像素尺寸大小;
输出:
    ret: 重投影误差;
    mtx: 内参矩阵;
    dist: 畸变系数;
    rvecs: 旋转向量 (外参数);
    tvecs: 平移向量 (外参数);
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    world_points, image_points, gray.shape[::-1], None, None)

# 保存相机参数(内参矩阵、畸变参数、旋转向量、平移向量)
np.savez('./Parameter/豪威OV48B.npz', mtx=mtx, dist=dist,
         rvecs=rvecs, tvecs=tvecs)  # 分别使用mtx,dist,rvecs,tvecs命名数组

print("ret(重投影误差):", ret,
      "\n\nmtx(内参矩阵):\n", mtx,
      "\n\ndist(畸变参数):\n", dist,  # 5个畸变参数,(k1,k2,p1,p2,k3)
      "\n\nrvecs(旋转向量):\n", rvecs,
      "\n\ntvecs(平移向量):\n", tvecs
      )

ret(重投影误差): 0.2991705035712761 

mtx(内参矩阵):
 [[596.34925072   0.         400.35710334]
 [  0.         596.49697012 300.47081541]
 [  0.           0.           1.        ]] 

dist(畸变参数):
 [[ 9.86732100e-02 -4.94565440e-01 -4.51708736e-04  2.89052555e-04
   8.22459145e-01]] 

rvecs(旋转向量):
 (array([[-0.0483275 ],
       [-0.05556281],
       [ 0.00805537]]), array([[-0.12482361],
       [ 0.15589444],
       [-0.07529847]]), array([[-0.17208193],
       [ 0.10014288],
       [-0.37814094]]), array([[-0.12937114],
       [ 0.11474067],
       [ 0.48904626]]), array([[0.30262844],
       [0.16153906],
       [0.03016236]]), array([[0.01218194],
       [0.56805057],
       [0.0150598 ]]), array([[-0.02910297],
       [-0.39260678],
       [-0.02031562]]), array([[-0.27239019],
       [-0.36934551],
       [ 0.12117097]]), array([[ 0.22883962],
       [-0.30550396],
       [-0.12688241]]), array([[ 0.06883582],
       [-0.27973277],
       [-0.06456626]]), array([[-0.2665373 ],
       [-0.185

### 获取Aruco码中心点的像素坐标

#### 方法原理
```
检测所拍摄的含有ArUco标记的图像,先获取每一个Aruco四个角点的像素坐标,随后计算出中心点的像素坐标,并保存为列表。
```

In [ ]:
# 导包
import os
import cv2
import numpy as np

In [ ]:
# 读取文件夹下所有图像
folder_path = './Aruco/'
image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')]

# 明确arUco字典和参数设置
aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_6X6_250)
# 使用默认值初始化检测器参数
aruco_params = cv2.aruco.DetectorParameters_create()

# 循环处理每张图像
aruco_centers_dict = {}
for image_path in image_paths:
    # 获取图像名称
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # 读取图像并转换为灰度图
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 检测arUco码
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

    # print(corners[i][0])
    # print(corners[i][0][0])
    # print(corners[i][0][0][0])
    # print(corners[i][0][0][1])
    
    # 保存arUco码的id和中心点像素坐标到字典中
    aruco_centers_dict[image_name] = {}
    if ids is not None:
        print(f"图像 {image_name} 中检测到 {len(ids)} 个arUco码,按照(arUco码的id,(arUco码的中心点像素坐标))的形式打印输出如下:")
        for i in range(len(ids)):
            # # 提取每个角点的横纵坐标
            # x_coords = corners[:, 0, :, 0]
            # y_coords = corners[:, 0, :, 1]
            # # 计算横坐标和纵坐标的均值
            # x_center = np.mean(x_coords)
            # y_center = np.mean(y_coords)
            # # 得到中心点的坐标
            # center = (x_center, y_center)
            center = (float((corners[i][0][0][0] + corners[i][0][2][0]) / 2.0), float((corners[i][0][0][1] + corners[i][0][2][1]) / 2.0))
            aruco_id = ids[i][0]
            aruco_centers_dict[image_name][aruco_id] = center
            print(f"({aruco_id},{center})")
    else:
        print(f"图像 {image_name} 中未检测到arUco码")

图像 IMG_20230425_171500 中检测到 6 个arUco码,按照(arUco码的id,(arUco码的中心点像素坐标))的形式打印输出如下:
(4,(405.0, 463.5))
(2,(366.0, 463.0))
(0,(327.0, 462.5))
(5,(403.5, 433.0))
(3,(366.5, 432.0))
(1,(329.5, 431.0))
图像 IMG_20230426_110858 中检测到 6 个arUco码,按照(arUco码的id,(arUco码的中心点像素坐标))的形式打印输出如下:
(4,(454.0, 486.5))
(2,(411.0, 484.0))
(0,(368.0, 481.0))
(5,(452.5, 452.5))
(3,(412.0, 450.0))
(1,(371.5, 447.0))
图像 IMG_20230426_110907 中检测到 6 个arUco码,按照(arUco码的id,(arUco码的中心点像素坐标))的形式打印输出如下:
(4,(547.0, 423.0))
(2,(507.0, 420.5))
(0,(467.5, 417.5))
(5,(539.5, 391.5))
(3,(502.0, 388.5))
(1,(464.0, 385.5))
图像 IMG_20230426_110911 中检测到 6 个arUco码,按照(arUco码的id,(arUco码的中心点像素坐标))的形式打印输出如下:
(0,(507.5, 400.0))
(2,(546.5, 398.0))
(4,(585.5, 395.5))
(1,(497.5, 370.0))
(3,(535.0, 368.0))
(5,(571.5, 365.5))


In [ ]:
# 要获取的照片名称和arUco码id
image_name = 'IMG_20230425_171500'  # 假设要获取该照片的id为0的arUco码的中心点坐标
aruco_id = 0

# 获取指定arUco码的中心点像素坐标
if image_name in aruco_centers_dict:
    if aruco_id in aruco_centers_dict[image_name]:
        target_aruco_center = aruco_centers_dict[image_name][aruco_id]
        print(f"照片 {image_name} 中arUco码 {aruco_id} 的中心点像素坐标为 {target_aruco_center}")
    else:
        print(f"照片 {image_name} 中未检测到id为 {aruco_id} 的arUco码。")
else:
    print(f"未找到名称为 {image_name} 的照片。")


照片 IMG_20230425_171500 中arUco码 0 的中心点像素坐标为 (327.0, 462.5)


### 根据两幅图像和图像上对应的四个点的三维世界坐标和二维像素坐标来求其他点的世界坐标
#### 要求已知
```
1.两幅图像上对应的四个点的世界坐标和像素坐标。
2.相机的内参矩阵和畸变系数。
```
#### 实现方法
```
使用立体视觉（stereo vision）的方法：首先计算两个相机的姿态（旋转和平移矩阵），然后使用这些矩阵和相机内参矩阵进行三角测量（triangulation）以获取其他点的世界坐标。
```

In [ ]:
# 导包
import cv2
import numpy as np

In [ ]:
# 使用PnP算法估计相机姿态
def estimate_camera_pose(world_coords, pixel_coords, camera_matrix, dist_coeffs):
    _, rvec, tvec, _ = cv2.solvePnPRansac(
        world_coords, pixel_coords, camera_matrix, dist_coeffs)
    return rvec, tvec

In [ ]:
# 三角测量计算世界坐标
def triangulate_points(pixel_coords1, pixel_coords2, camera_matrix, rvec1, tvec1, rvec2, tvec2):
    proj_matrix1 = np.dot(camera_matrix, np.hstack(
        (cv2.Rodrigues(rvec1)[0], tvec1)))
    proj_matrix2 = np.dot(camera_matrix, np.hstack(
        (cv2.Rodrigues(rvec2)[0], tvec2)))
    world_coords = cv2.triangulatePoints(
        proj_matrix1, proj_matrix2, pixel_coords1.T, pixel_coords2.T).T
    world_coords = (world_coords / world_coords[:, 3:])[:, :3]
    return world_coords

In [ ]:
# 两幅图像中对应的世界坐标和像素坐标
world_coords_list = [
    # 图像1的四个点对应的世界坐标
    np.array([[-50, -50, 0], 
              [-50, 50, 0], 
              [50, -50, 0],
             [50, 50, 0]], dtype=np.float32),
    # 图像2的四个点对应的世界坐标
    np.array([[-50, -50, 0], 
              [-50, 50, 0], 
              [50, -50, 0],
             [50, 50, 0]], dtype=np.float32)
]

pixel_coords_list = [
    # 图像1的四个点对应的像素坐标
    np.array([[327.0, 462.5],
              [329.5, 431.0],
              [366.0, 463.0],
             [366.5, 432.0]], dtype=np.float32),
    # 图像2的四个点对应的像素坐标
    np.array([[368.0, 481.0], 
              [371.5, 447.0], 
              [411.0, 484.0], 
              [412.0, 450.0]], dtype=np.float32)
]

In [ ]:
# 加载npz文件读取相机的内参矩阵和畸变系数
data = np.load('./Parameter/豪威OV48B.npz') 

# 获取mtx和dist参数
camera_matrix = data['mtx']
dist_coeffs = data['dist']

# 计算两幅图像的相机姿态
camera_poses = [estimate_camera_pose(world_coords, pixel_coords, camera_matrix, dist_coeffs)
                for world_coords, pixel_coords in zip(world_coords_list, pixel_coords_list)]

# 获取每幅图像的旋转向量rvec和平移向量tvec
rvec1, tvec1 = camera_poses[0]
rvec2, tvec2 = camera_poses[1]

In [ ]:
def pixel_to_world(pixel_coord_1,pixel_coord_2):
    world_coord = triangulate_points(
        np.array([pixel_coord_1], dtype=np.float32),
        np.array([pixel_coord_2], dtype=np.float32),
        camera_matrix, rvec1, tvec1, rvec2, tvec2
    )[0]
    print(world_coord)

pixel_to_world((405.0, 463.5),(454.0, 486.5))
pixel_to_world((403.5, 433.0),(452.5, 452.5))

[172.93314  -57.318256   4.613435]
[168.27997    41.85213     5.7491765]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

In [ ]:
# 监测到的多个点的坐标变化数据
# 每个数组包含相应坐标分量的前后值
points = [
    {'before': [-50, -50, 0], 'after': [-70, 60, 13]},
    {'before': [-50, 50, 0], 'after': [-40, 30, 20]},
    {'before': [50, -50, 0], 'after': [60, -80, 90]},
    {'before': [50, 50, 0], 'after': [20, 50, 33]},
]

In [ ]:
# 创建3D箭头图
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 绘制箭头
for point in points:
    x_before, y_before, z_before = point['before']
    x_after, y_after, z_after = point['after']
    dx = x_after - x_before
    dy = y_after - y_before
    dz = z_after - z_before
    ax.quiver(x_before, y_before, z_before, dx, dy,
              dz, color='b', arrow_length_ratio=0.1)

# 设置坐标轴标签
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

# 设置标题
ax.set_title('多点世界坐标(X,Y,Z)变化的3D箭头图')

# 设置坐标轴范围
all_x = [point['before'][0]
         for point in points] + [point['after'][0] for point in points]
all_y = [point['before'][1]
         for point in points] + [point['after'][1] for point in points]
all_z = [point['before'][2]
         for point in points] + [point['after'][2] for point in points]
ax.set_xlim(min(all_x), max(all_x))
ax.set_ylim(min(all_y), max(all_y))
ax.set_zlim(min(all_z), max(all_z))

# 显示图形
plt.show()